# Create BQ Dataset Reporting

In [2]:
reporting_dataset_id = "MIT_reporting"
!bq --location=US mk --dataset {reporting_dataset_id}

Dataset 'sound-cider-252823:MIT_reporting' successfully created.


# Create View for Query 1

This query returns the names of top 10 candidates who gain the highest amount of votes in Texas.

In [5]:
%%bigquery
Create View MIT_reporting.v_Highest_Votes_Candidates as
select distinct c.Name, r.Candidate_Votes, e.Year, e.Type
from (`sound-cider-252823.fec_modeled.Candidates_Beam_DF_Jupyter` c 
inner join `sound-cider-252823.MIT_modeled.Runs_Beam_DF_Jupyter` r
on c.Label = r.Candidate_Label)
inner join `sound-cider-252823.MIT_modeled.Elections` e 
on e.Election_ID = r.Election_ID
where e.State = "TX"
order by r.Candidate_Votes DESC
limit 10

""


# Create View for Query 2

This query returns the names of top 10 candidates who receive the highest amount of money in a single contribution. This query also shows the contribution committee and contribution amount. 

In [7]:
%%bigquery
Create View MIT_reporting.v_Highest_Contribution_Candidates as
select distinct c.Name as Candidate_Name, e.Type, con.Amount, e.Year, com.Name as Committee_Name
from (((`sound-cider-252823.fec_modeled.Candidates_Beam_DF_Jupyter` c 
inner join `sound-cider-252823.MIT_modeled.Runs_Beam_DF_Jupyter` r
on c.Label = r.Candidate_Label)
inner join `sound-cider-252823.fec_modeled.Contributes_Beam_DF` con
on con.Candidate_Label = c.Label)
inner join `sound-cider-252823.MIT_modeled.Elections` e
on e.Election_ID = r.Election_ID)
inner join `sound-cider-252823.fec_modeled.Committees_Beam_DF` com
on con.Committee_Label = com.Label
order by con.Amount DESC
limit 10



""


# Create View for Query 3

This query returns the top 10 states which give the highest amount of contribution in year 2016 and the number of votes in 2016 presidential election in that state. This query aim to find whether there is a relationship between the total political contribution and the total number of votes.

In [12]:
%%bigquery
Create View MIT_reporting.v_States_with_Top_Contribution as
select t.State, t.Total_Contribution, e.Total_Votes
from
(select c.State as State, sum(con.Amount) as Total_Contribution
from (`sound-cider-252823.fec_modeled.Contributes_Beam_DF` con
inner join `sound-cider-252823.fec_modeled.Committees_Beam_DF` c
on c.Label = con.Committee_Label)
where c.Year = 2016
group by c.State) as t
inner join `sound-cider-252823.MIT_modeled.Elections` e
on t.State = e.State
where e.Type = "President" and e.Year = 2016
order by Total_Contribution DESC

""
